In [5]:
# datasetPath = r"E:\Semester 6\AI\Datasets\2\Original Images"
datasetPath = r"face recognition 2\Original Images"
# image = r"E:\Semester 6\AI\Datasets\2\Original Images\Original Images\Akshay Kumar\Akshay Kumar_1.jpg"

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install mtcnn

In [ ]:
pip install tensorflow

In [ ]:
pip install matplotlib

In [10]:
import os
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [11]:
# img = cv.imread("/content/drive/MyDrives/AI Proj/face recognition/Face Data/Face Dataset/0/0.jpg")
# img = cv.imread("/content/drive/MyDrive/AI Project Dataset/0/0.jpg")
image = r"face recognition 2\Original Images\Akshay Kumar\Akshay Kumar_1.jpg"
inputImage = cv.imread(image)
# inputImage = cv.imread("/content/drive/MyDrive/AI Proj/face recognition 2/Original Images/Akshay Kumar/Akshay Kumar_1.jpg")

In [ ]:
inputImage = cv.cvtColor(inputImage, cv.COLOR_BGR2RGB)
plt.imshow(inputImage) #changed to RGB bcz openCV works on RGB

In [ ]:
from mtcnn.mtcnn import MTCNN

imageDetector = MTCNN()
firstResults = imageDetector.detect_faces(inputImage)

In [ ]:
firstResults

In [15]:
i, j, k, l= firstResults[0]['box'] #only need box x,y,w,h 

In [ ]:
inputImage = cv.rectangle(inputImage, (i,j), (i+k, j+l), (255,255,0), 30)
plt.imshow(inputImage)

In [ ]:
firstFace = inputImage[j:j+l, i:i+k]
firstFace = cv.resize(firstFace, (160,160)) # as Facenet needs input in 160x160 
plt.imshow(firstFace)

In [ ]:
firstFace

##Automating the preprocessing of the image: 
Input image, BGR to RGB, resizing, rectangle box around face

In [19]:
class AutomaticLoadingFace:
        #putting all images in x array and all labels in y array
    def __init__(self, folder):  
        self.folder = folder
        self.targetSize = (160,160)
        self.xaxis = []
        self.yaxis = []
        self.imageDetector = MTCNN()
    

    def extractFace(self, file):
        inputImage = cv.imread(file)
        inputImage = cv.cvtColor(inputImage, cv.COLOR_BGR2RGB)
        i, j, k, l = self.imageDetector.detect_faces(inputImage)[0]['box']
        i, j = abs(i), abs(j)
        firstFace = inputImage[j:j+l, i:i+k]
        firstFaceArray = cv.resize(firstFace, self.targetSize)
        return firstFaceArray
    

    def loadFaces(self, directory):
        FACES = []
        for iterator in os.listdir(directory):
            try:
                path = directory + iterator
                singleFace = self.extractFace(path)
                FACES.append(singleFace)
            except Exception as e:
                pass
        return FACES

    def loadClasses(self):
        for subDirectory in os.listdir(self.folder):
            path = self.folder +'/'+ subDirectory+'/'
            FACES = self.loadFaces(path)
            imageLabels = [subDirectory for _ in range(len(FACES))]
            print(f"Loaded successfully: {len(imageLabels)}")
            self.xaxis.extend(FACES)
            self.yaxis.extend(imageLabels)
        
        return np.asarray(self.xaxis), np.asarray(self.yaxis)


    def plotImages(self):
        plt.figure(figsize=(18,16))
        for number, iImage in enumerate(self.xaxis):
            noOfCols = 3
            noOfRows = len(self.yaxis)//noOfCols + 1
            plt.subplot(noOfRows, noOfCols, number+1)
            plt.imshow(iImage)
            plt.axis('off')

In [ ]:
# faceloading = FACELOADING("/content/drive/MyDrive/AI Proj/face recognition 2")
# f = r"E:\Semester 6\AI\Datasets\2\Original Images\Original Images"
f = r"face recognition 2\Original Images"
faceLoading = AutomaticLoadingFace(f)
xaxis, yaxis = faceLoading.loadClasses()

In [ ]:
# # faceloading = FACELOADING("/content/drive/MyDrive/AI Proj/face recognition 2")
# # f = r"E:\Semester 6\AI\Datasets\2\Original Images\Original Images"
# f = r"face recognition 2\Original Images"
# faceLoading = AutomaticLoadingFace(f)
# xaxis, yaxis = faceLoading.loadClasses()

In [ ]:
# plt.figure(figsize=(144, 144))

plt.figure(figsize=(216, 216))
for number, iImage in enumerate(xaxis):
    noOfCols = 31
    noOfRows = len(yaxis)//noOfCols + 1
    plt.subplot(noOfRows, noOfCols, number+1)
    plt.imshow(iImage)
    plt.axis('off')

##FAMMA-FaceNet


In [ ]:
!pip install keras-facenet

In [27]:
from keras_facenet import FaceNet
imageEmbedder = FaceNet()

def getEmbedding(faceImage):
  faceImage = faceImage.astype("float32") #3Dimensional(160x160x3)
  faceImage = np.expand_dims(faceImage, axis=0) #need 4Dimensional(Nonex160x160x3)
  oneFace = imageEmbedder.embeddings(faceImage)
  return oneFace[0] #will return 512D image (1x1x512)


In [ ]:
embeddedX = []
for ik in xaxis:
  embeddedX.append(getEmbedding(ik))

embeddedX = np.asarray(embeddedX)

In [30]:
np.savez_compressed('EmbeddingsForFaceImages.npz', embeddedX, yaxis)

#SVM model
compare 512dimesional arrays to each other and how theyre working

In [ ]:
pip install -U scikit-learn

In [32]:
# yaxis #checking

from sklearn.preprocessing import LabelEncoder

encodingLabels = LabelEncoder()
encodingLabels.fit(yaxis)
yaxis = encodingLabels.transform(yaxis)

In [ ]:
plt.plot(embeddedX[0]) 
plt.ylabel(yaxis[0])

In [ ]:
yaxis

In [35]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(embeddedX, yaxis, shuffle=True, random_state=17)
#shuffling so it doesnt just train on face of 1 person

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainY)

In [37]:
YPredictTrain = model.predict(trainX)
YPredictTest = model.predict(testX)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(trainY, YPredictTrain)

In [ ]:
accuracy_score(testY, YPredictTest)

##Testing

In [ ]:
# TestImage = cv.imread("/content/drive/MyDrive/AI Proj/face recognition 2/Faces/Akshay Kumar_13.jpg")
ij = r"face recognition 2/Faces/Akshay Kumar_13.jpg"
TestImage = cv.imread(ij)

TestImage = cv.cvtColor(TestImage, cv.COLOR_BGR2RGB)
i, j, k, l = imageDetector.detect_faces(TestImage)[0]['box']

In [ ]:
TestImage = TestImage[j:j+l, i:i+k]
TestImage = cv.resize(TestImage, (160,160)) 
TestImageEmbedding = getEmbedding(TestImage)


In [42]:
TestImageEmbedding = [TestImageEmbedding]
YPredict = model.predict(TestImageEmbedding)

In [ ]:
YPredict

In [ ]:
encodingLabels.inverse_transform(YPredict)

In [45]:
#saving this svm model
import pickle

with open('SVM_Model(160x160).pkl','wb') as f:
    pickle.dump(model,f)